In [1]:
# %pip install bertopic
# %pip install indicnlp-library
# %pip install transformers


In [2]:
# get processed data

import os
import pandas as pd

file_path = "../data/"

data = pd.read_csv(file_path+"train.csv")



In [3]:
# Convert to list
df=data["news_title"]

docs = [str(i) for i in df.values]

docs=docs[:10000]

In [4]:


# Use model to convert words to vectors - EMBEDDING STEP

import transformers
import numpy as np
from tqdm import tqdm
import torch 

from transformers.pipelines import pipeline


model_name = "ai4bharat/indic-bert"

# Use tokenizer to separate words

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, keep_accents=True)

# Use model to convert words to vectors - EMBEDDING STEP

model = pipeline("feature-extraction", model=model_name, tokenizer=tokenizer)



/Users/samyuktha/opt/anaconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identic

In [5]:
# Tokenize docs trivially (split on spaces)

from indicnlp.tokenize import sentence_tokenize, indic_tokenize

def tokenize_ta(text,return_tensors="pt",*args,**kwargs):
    return indic_tokenize.trivial_tokenize(text)

In [6]:
# Common list of stopwords

stopwords=['அங்கு',
 'அங்கே',
 'அடுத்த',
 'அதனால்',
 'அதன்',
 'அதற்கு',
 'அதிக',
 'அதில்',
 'அது',
 'அதே',
 'அதை',
 'அந்த',
 'அந்தக்',
 'அந்தப்',
 'அன்று',
 'அல்லது',
 'அவன்',
 'அவரது',
 'அவர்',
 'அவர்கள்',
 'அவள்',
 'அவை',
 'ஆகிய',
 'ஆகியோர்',
 'ஆகும்',
 'இங்கு',
 'இங்கே',
 'இடத்தில்',
 'இடம்',
 'இதனால்',
 'இதனை',
 'இதன்',
 'இதற்கு',
 'இதில்',
 'இது',
 'இதை',
 'இந்த',
 'இந்தக்',
 'இந்தத்',
 'இந்தப்',
 'இன்னும்',
 'இப்போது',
 'இரு',
 'இருக்கும்',
 'இருந்த',
 'இருந்தது',
 'இருந்து',
 'இவர்',
 'இவை',
 'உன்',
 'உள்ள',
 'உள்ளது',
 'உள்ளன',
 'எந்த',
 'என',
 'எனக்',
 'எனக்கு',
 'எனப்படும்',
 'எனவும்',
 'எனவே',
 'எனினும்',
 'எனும்',
 'என்',
 'என்ன',
 'என்னும்',
 'என்பது',
 'என்பதை',
 'என்ற',
 'என்று',
 'என்றும்',
 'எல்லாம்',
 'ஏன்',
 'ஒரு',
 'ஒரே',
 'ஓர்',
 'கொண்ட',
 'கொண்டு',
 'கொள்ள',
 'சற்று',
 'சிறு',
 'சில',
 'சேர்ந்த',
 'தனது',
 'தன்',
 'தவிர',
 'தான்',
 'நான்',
 'நாம்',
 'நீ',
 'பற்றி',
 'பற்றிய',
 'பல',
 'பலரும்',
 'பல்வேறு',
 'பின்',
 'பின்னர்',
 'பிற',
 'பிறகு',
 'பெரும்',
 'பேர்',
 'போது',
 'போன்ற',
 'போல',
 'போல்',
 'மட்டுமே',
 'மட்டும்',
 'மற்ற',
 'மற்றும்',
 'மிக',
 'மிகவும்',
 'மீது',
 'முதல்',
 'முறை',
 'மேலும்',
 'மேல்',
 'யார்',
 'வந்த',
 'வந்து',
 'வரும்',
 'வரை',
 'வரையில்',
 'விட',
 'விட்டு',
 'வேண்டும்',
 'வேறு']

from sklearn.feature_extraction.text import CountVectorizer

# Create a vectorizer object to generate term document counts for topic representation - TOKENIZATION STEP

vectorizer_model = CountVectorizer(
    stop_words=stopwords,analyzer='word',
    tokenizer=tokenize_ta
)

In [7]:
import pandas as pd
from scipy.sparse import csr_matrix
from typing import List, Mapping, Tuple, Union
import stanza

class TamilPOS():
    """
    Extract Topic Keywords based on their Part-of-Speech using stanza library for Tamil.
    """
    def __init__(self,
                 top_n_words: int = 10,
                 pos_patterns: List[str] = None):
        self.top_n_words = top_n_words

        if pos_patterns is None:
            self.pos_patterns = [
                'NOUN',
                'PROPN',
            ]
        else:
            self.pos_patterns = pos_patterns

        # load stanza pipeline for Tamil
        self.nlp = stanza.Pipeline(lang='ta', processors='tokenize,pos')

    def extract_topics(self,
                       topic_model,
                       documents: pd.DataFrame,
                       c_tf_idf: csr_matrix,
                       topics: Mapping[str, List[Tuple[str, float]]]
                       ) -> Mapping[str, List[Tuple[str, float]]]:
        topic_to_keywords = {}
        for topic_id, topic_words in topics.items():
            # filter candidate documents that contain at least one keyword from the topic
            mask = documents['text'].str.contains('|'.join([word[0] for word in topic_words]), regex=True)
            candidate_docs = documents[mask]

            # extract candidate keywords from candidate_docs based on POS patterns
            candidate_keywords = []
            for doc in candidate_docs['text']:
                doc_keywords = []
                # get POS tags for each word in the document
                doc_words = self.nlp(doc).sentences[0].words
                for word in doc_words:
                    if word.upos in self.pos_patterns:
                        doc_keywords.append(word.text)
                candidate_keywords.extend(doc_keywords)
            # count the frequency of each keyword and keep the top n
            candidate_keyword_counts = pd.Series(candidate_keywords).value_counts().head(self.top_n_words)
            # normalize keyword counts
            candidate_keyword_counts = candidate_keyword_counts / candidate_docs.shape[0]
            # assign c-TF-IDF scores to keywords
            keyword_scores = [(word, topic_model.get_topic(topic_id)[word]) for word in candidate_keyword_counts.index]
            # sort keywords by their respective c-TF-IDF scores
            sorted_keyword_scores = sorted(keyword_scores, key=lambda x: x[1], reverse=True)
            # add top n keywords to topic_to_keywords dict
            topic_to_keywords[topic_id] = sorted_keyword_scores[:self.top_n_words]
        return topic_to_keywords


In [8]:
representation_model = TamilPOS()

2023-03-24 09:23:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-03-24 09:23:09 WARNING: Language ta package default expects mwt, which has been added
2023-03-24 09:23:10 INFO: Loading these models for language: ta (Tamil):
| Processor | Package |
-----------------------
| tokenize  | ttb     |
| mwt       | ttb     |
| pos       | ttb     |

2023-03-24 09:23:10 INFO: Using device: cpu
2023-03-24 09:23:10 INFO: Loading: tokenize
2023-03-24 09:23:10 INFO: Loading: mwt
2023-03-24 09:23:10 INFO: Loading: pos
2023-03-24 09:23:10 INFO: Done loading processors!


In [9]:

# Create a BERTopic model

from bertopic import BERTopic

topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    verbose=True,
    calculate_probabilities=False,
    embedding_model=model,
    representation_model=representation_model
    
)




In [10]:
topics = topic_model.fit_transform(docs)


100%|██████████| 10000/10000 [16:12<00:00, 10.29it/s]  
2023-03-24 09:39:25,292 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2023-03-24 09:39:42,644 - BERTopic - Reduced dimensionality
2023-03-24 09:39:42,864 - BERTopic - Clustered reduced embeddings


In [11]:
# Get the topics

topic_model.get_topic_info()



,Topic,Count,Name
0,-1,4463,-1_கைது_பலி_தேர்தல்_அரசு
1,0,588,0_இந்தியா_உத்தரவு_வழக்கு_கொள்ளை
2,1,374,1_தவிப்பு_காங்கிரஸ்_நடவடிக்கை_கல்வி
3,2,362,2_கொள்ளை_ஐகோர்ட்_பறிமுதல்_பசிதம்பரம்
4,3,361,3_வாக்குப்பதிவு_பிடிபட்டனர்_சிக்கினர்_மாயம்
...,...,...,...
71,70,12,70_மோதி_பலி_பைக்_மரத்தில்
72,71,12,71_லாரி_மோதி_நசுங்கி_பலி
73,72,11,72_எம்சிஏ_யுபிஎஸ்சி_படிக்கலாம்_நடத்தும்
74,73,10,73_கூவி_பழம்_அில்_முதுகுல


In [12]:
# Get the words in a topic

topic_model.get_topic(0)


[('இந்தியா', 0.012164576658192103),
 ('உத்தரவு', 0.01123240627397045),
 ('வழக்கு', 0.010940998417797262),
 ('கொள்ளை', 0.010428276089279),
 ('கண்டனம்', 0.010340407622108593),
 ('நிறுத்தம்', 0.009870173525594569),
 ('பணி', 0.009481166016287856),
 ('கோரி', 0.008964155341095582),
 ('வீழ்த்தியது', 0.008526734173224741),
 ('தற்கொலை', 0.008126713728246733)]

In [13]:
topic_model.get_representative_docs(0)


['ராணுவ கேப்டன் வீட்டில் நகை பணம் கொள்ளை',
 'பிறந்த நாள் பரிசு வழக்கில் விடுவிக்க கோரி ஜெ மனு விசாரணை மீண்டும் தள்ளிவைப்பு',
 'சமச்சீர் கல்வி பாட புத்தகங்கள் அனுப்பும் பணி திடீர் நிறுத்தம்']

In [14]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents
cleaned_docs = topic_model._preprocess_text(docs)

# Extract vectorizer and tokenizer from BERTopic
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(topic_model.get_topics())-1)]


In [15]:

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

0.40981318853955884

In [16]:
topic_model.visualize_topics()

In [17]:
topic_model.visualize_barchart()